In [15]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [16]:
pd.options.display.max_columns = 999

In [17]:
claims_df = pd.read_csv("data/claims.csv")

claims_features= claims_df.loc[:, ("dateOfLoss", 
                          "baseFloodElevation", 
                          "ratedFloodZone", 
                          "locationOfContents", 
                          "occupancyType", 
                          "amountPaidOnBuildingClaim",
                          "amountPaidOnContentsClaim",
                          "totalBuildingInsuranceCoverage",
                          "totalContentsInsuranceCoverage",
                          "yearOfLoss",
                          "primaryResidenceIndicator",
                          "buildingDamageAmount", 
                          "netBuildingPaymentAmount",
                          "buildingPropertyValue",
                          "contentsDamageAmount",
                          "netContentsPaymentAmount",
                          "contentsPropertyValue",
                          "floodCharacteristicsIndicator",
                          "floodWaterDuration",
                          "floodproofedIndicator",
                          "floodEvent",
                          "buildingReplacementCost",
                          "contentsReplacementCost",
                          "stateOwnedIndicator",
                          "buildingDescriptionCode",
                          "rentalPropertyIndicator",
                          "state",
                          "countyCode",
                          "censusTract",
                          "censusBlockGroupFips",
                          "latitude",
                          "longitude",
                          "id")]

claims_features['countyCode'] = claims_features['countyCode'].astype(str).str.rstrip('.0')

claims_features['censusTract'] = claims_features['censusTract'].astype(str).str.rstrip('.0')

claims_features['censusBlockGroupFips'] = claims_features['censusBlockGroupFips'].astype(str).str.rstrip('.0')



# Remove rows with missing geometry
claims_features = claims_features.dropna(subset=["latitude", "longitude"])


# Create our GeoDataFrame with geometry column created from lon/lat
claims = gpd.GeoDataFrame(
    claims_features,
    geometry=gpd.points_from_xy(claims_df["longitude"], claims_df["latitude"]),
    crs="EPSG:4326",
)

claims_gpd = claims.to_crs('EPSG:3857')

claims_gpd['x'] = claims_gpd['geometry'].x
claims_gpd['y'] = claims_gpd['geometry'].y


C:\Users\dtdre\AppData\Local\Temp\ipykernel_7604\4081295779.py:1: DtypeWarning: Columns (8,36,39,43,52,53,62) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv("data/claims.csv")


In [18]:
states = ['NJ']

claims_NJ = claims_gpd[(claims_gpd['yearOfLoss'] >= 1995) & (claims_gpd['state'].isin(states))]

In [19]:
from pathlib import Path  

filepath = Path('data/claims_NJ.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

claims_NJ.to_csv(filepath) 

In [20]:
claims_NJ_clean=claims_NJ

sfh = (claims_NJ_clean['occupancyType'] == 1) | (claims_NJ_clean['occupancyType'] == 11)
claims_NJ_clean['sfh'] = np.where(sfh, 1, 0)

claims_NJ_clean['observation'] = 1

claims_sfh = claims_NJ_clean[claims_NJ_clean['sfh'] == 1]

C:\Users\dtdre\mambaforge\envs\musa-550-fall-2023\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\dtdre\mambaforge\envs\musa-550-fall-2023\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\dtdre\mambaforge\envs\musa-550-fall-2023\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on 

In [24]:
claims_sfh

,dateOfLoss,baseFloodElevation,ratedFloodZone,locationOfContents,occupancyType,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,netBuildingPaymentAmount,buildingPropertyValue,contentsDamageAmount,netContentsPaymentAmount,contentsPropertyValue,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,buildingReplacementCost,contentsReplacementCost,stateOwnedIndicator,buildingDescriptionCode,rentalPropertyIndicator,state,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id,geometry,x,y,sfh,observation
1,2012-10-29T00:00:00.000Z,NaN,AE,NaN,1.0,55000.00,0.0,55000.0,0.0,2012,1,67506.0,55000.00,151170.0,NaN,0.0,NaN,NaN,0.0,0,Hurricane Sandy,215958.0,NaN,0,1.0,0,NJ,34029,34029729,340297290002,39.9,-74.1,0157bf63-f25f-47bc-a9a8-6ef027f36130,POINT (-8248774.268 4851421.175),-8.248774e+06,4.851421e+06,1,1
42606,2012-10-29T00:00:00.000Z,11.0,AE,7.0,1.0,12979.08,1630.0,76200.0,6400.0,2012,1,10933.0,12979.08,72352.0,2630.0,1630.0,0.0,NaN,0.0,0,Hurricane Sandy,161587.0,0.0,0,1.0,0,NJ,34001,340010102,340010102002,39.4,-74.5,36c33c4f-4f93-4941-a6ae-7555c3aa5bea,POINT (-8293302.064 4779131.181),-8.293302e+06,4.779131e+06,1,1
42609,2012-10-29T00:00:00.000Z,NaN,AE,2.0,1.0,17530.18,5000.0,120000.0,5000.0,2012,1,19530.0,17530.18,178560.0,10014.0,5000.0,0.0,NaN,0.0,0,Hurricane Sandy,223200.0,0.0,0,1.0,0,NJ,34001,340010103,340010103001,39.4,-74.5,9630691c-44ec-4d3f-8a7d-6ae2a2167cbf,POINT (-8293302.064 4779131.181),-8.293302e+06,4.779131e+06,1,1
42610,2012-10-29T00:00:00.000Z,9.0,A05,NaN,1.0,77703.62,2387.7,250000.0,54300.0,2012,0,79704.0,77703.62,217194.0,4388.0,2387.7,0.0,NaN,0.0,0,Hurricane Sandy,293505.0,0.0,0,NaN,0,NJ,34001,340010116,340010116002,39.3,-74.8,74923e49-ff47-48e1-b75f-1f491bc05025,POINT (-8326697.911 4764735.556),-8.326698e+06,4.764736e+06,1,1
42611,2012-10-29T00:00:00.000Z,7.7,AE,7.0,1.0,21677.50,1216.4,225000.0,10000.0,2012,1,22677.0,21677.50,177715.0,2216.0,1216.4,0.0,NaN,0.0,0,Hurricane Sandy,214115.0,0.0,0,1.0,0,NJ,34001,340076103,340076103001,39.9,-75.1,adcbabd5-7e84-4fd4-8edc-d2f0f27532a6,POINT (-8360093.759 4851421.175),-8.360094e+06,4.851421e+06,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473602,1999-09-16T00:00:00.000Z,NaN,X,NaN,1.0,163982.11,34800.0,250000.0,34800.0,1999,0,164482.0,163982.11,235000.0,35300.0,34800.0,0.0,NaN,0.0,0,Hurricane Floyd,250000.0,0.0,0,NaN,0,NJ,34035,34035053604,340350536042,40.6,-74.7,a3fcb282-9d4a-4258-8a44-636cfe55348d,POINT (-8315565.962 4953519.587),-8.315566e+06,4.953520e+06,1,1
2473603,2012-10-29T00:00:00.000Z,NaN,A10,NaN,1.0,13030.26,0.0,250000.0,0.0,2012,1,16031.0,13030.26,224595.0,NaN,0.0,NaN,NaN,0.0,0,Hurricane Sandy,320850.0,NaN,0,1.0,0,NJ,34035,340350518,340350518002,40.6,-74.4,0018e98e-22fe-4bb6-b10f-c0fe98e6c306,POINT (-8282170.115 4953519.587),-8.282170e+06,4.953520e+06,1,1
2473604,2007-04-15T00:00:00.000Z,NaN,A10,NaN,1.0,7502.63,0.0,200000.0,0.0,2007,1,10504.0,7502.63,173107.0,NaN,0.0,NaN,NaN,0.0,0,Nor'easter,216383.0,NaN,0,1.0,0,NJ,34035,340350518,340350518002,40.6,-74.4,d3c7acd3-7c4e-4774-b2a6-2ac4587c89a1,POINT (-8282170.115 4953519.587),-8.282170e+06,4.953520e+06,1,1
2473605,2011-08-28T00:00:00.000Z,NaN,X,2.0,1.0,1979.98,0.0,194700.0,15800.0,2011,1,3980.0,1979.98,115527.0,NaN,0.0,NaN,NaN,0.0,0,Hurricane Irene,144408.0,NaN,0,1.0,0,NJ,34035,340350512,340350512003,40.6,-74.5,8037c27d-9a9e-4e78-a8d5-a2ce8d84baa9,POINT (-8293302.064 4953519.587),-8.293302e+06,4.953520e+06,1,1


In [21]:
from pathlib import Path  

filepath = Path('data/claims_NJ_clean.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

claims_NJ_clean.to_csv(filepath) 

In [22]:
from pathlib import Path  

filepath = Path('data/claims_sfh.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

claims_sfh.to_csv(filepath) 